In [1]:
import numpy as np
import pandas as pd

In [2]:
resaleflat1 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
#resaleflat2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
resaleflat3 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
resaleflat4 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
resaleflat = pd.concat((resaleflat1,resaleflat3,resaleflat4), axis=0, sort=True)

resaleflat_l = resaleflat.copy()
resaleflatview = resaleflat1.copy()

In [3]:
resaleflat.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [4]:
resaleflat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139788 entries, 0 to 37152
Data columns (total 11 columns):
block                  139788 non-null object
flat_model             139788 non-null object
flat_type              139788 non-null object
floor_area_sqm         139788 non-null float64
lease_commence_date    139788 non-null int64
month                  139788 non-null object
remaining_lease        87585 non-null object
resale_price           139788 non-null float64
storey_range           139788 non-null object
street_name            139788 non-null object
town                   139788 non-null object
dtypes: float64(2), int64(1), object(8)
memory usage: 12.8+ MB


In [5]:
resaleflat.isnull().sum()

#Remaining lease has a lot of null, need to fix

block                      0
flat_model                 0
flat_type                  0
floor_area_sqm             0
lease_commence_date        0
month                      0
remaining_lease        52203
resale_price               0
storey_range               0
street_name                0
town                       0
dtype: int64

# We had to convert categorical to numerical first to see correlations, now we redo the table using onehotencoding

In [6]:
resaleflat_l.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [7]:
#Convert first two char into integer for years left
resaleflat_l['remaining_lease_year'] = resaleflat_l['remaining_lease'].str.slice(stop=2).astype('float')
#Substring/slice the month out
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease'].str.slice(start=9,stop=12)

replacemonth = {}
replacemonth[''] = 0
replacemonth['0 m'] = 0
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].replace(replacemonth)

resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].astype('float') / 12
resaleflat_l['remaining_lease'] = resaleflat_l['remaining_lease_year'] + resaleflat_l['remaining_lease_month']
resaleflat_l = resaleflat_l.drop(['remaining_lease_year','remaining_lease_month'], axis=1)

resaleflat_l.month = resaleflat_l.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
resaleflat_l.remaining_lease[resaleflat_l.remaining_lease.isnull()] = resaleflat_l['month'] - resaleflat_l['lease_commence_date']

#Normalize the month, floor area and remaining lease (timeframe)
resaleflat_l.month = (resaleflat_l.month - resaleflat_l.month.mean()) / resaleflat_l.month.std()
resaleflat_l.floor_area_sqm = (resaleflat_l.floor_area_sqm - resaleflat_l.floor_area_sqm.mean()) / resaleflat_l.floor_area_sqm.std()
resaleflat_l.remaining_lease = (resaleflat_l.remaining_lease - resaleflat_l.remaining_lease.mean()) / resaleflat_l.remaining_lease.std()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#Blocks in Singapore is rather irregular and does not impact on the pricing
#Lease commence date is highly correlated with remaining lease as remaining lease = current date - lease commence date

#Try using street name (high variation) instead of town now, both are highly correlated so just use one

resaleflat_l = resaleflat_l.drop(columns=['block','lease_commence_date','town','flat_type'])

In [9]:
resaleflat_l.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,street_name
0,Improved,-2.163608,0.540959,0.710285,232000.0,10 TO 12,ANG MO KIO AVE 10
1,New Generation,-1.225543,0.540959,0.681966,250000.0,01 TO 03,ANG MO KIO AVE 4
2,New Generation,-1.225543,0.540959,0.751191,262000.0,01 TO 03,ANG MO KIO AVE 5
3,New Generation,-1.184757,0.540959,0.738605,265000.0,04 TO 06,ANG MO KIO AVE 10
4,New Generation,-1.225543,0.540959,0.751191,265000.0,01 TO 03,ANG MO KIO AVE 5


In [10]:
all_flat_model = pd.get_dummies(resaleflat_l['flat_model'], prefix='flt')
all_streets = pd.get_dummies(resaleflat_l['street_name'], prefix='st')

resaleflat_l = resaleflat_l.drop(columns=['flat_model','street_name'])
resaleflat_l = pd.concat((resaleflat_l,all_flat_model,all_streets), axis=1)

In [11]:
resaleflat_l.storey_range.unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45', '06 TO 10', '01 TO 05', '11 TO 15',
       '16 TO 20', '21 TO 25', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [12]:
#Get average of storey range e.g. 01 to 03 would be 2
resaleflat_l.storey_range = resaleflat_l.storey_range.apply(lambda x: (float(x[0:2]) + float(x[7:9]) ) /2 )

resaleflat_l.storey_range = (resaleflat_l.storey_range - resaleflat_l.storey_range.mean()) / resaleflat_l.storey_range.std()

In [13]:
storey_resales = resaleflat_l[['storey_range','resale_price']]
storey_resales.corr()

#Storey range has correlation of 0.30 with resale price

,storey_range,resale_price
storey_range,1.000000,0.303071
resale_price,0.303071,1.000000


In [14]:
lease_resales = resaleflat_l[['remaining_lease','resale_price']]
lease_resales.corr()

#Remaining lease only has -0.067 correlation with resale price

,remaining_lease,resale_price
remaining_lease,1.0000,-0.0671
resale_price,-0.0671,1.0000


In [15]:
month_resales = resaleflat_l[['month','resale_price']]
month_resales.corr()

#Year/month bought (month) has -0.07 correlation with resale price

,month,resale_price
month,1.000000,-0.073186
resale_price,-0.073186,1.000000


In [16]:
floor_resales = resaleflat_l[['floor_area_sqm','resale_price']]
floor_resales.corr()

#Floor area SQM has 0.667 correlation with resale price, very high!

,floor_area_sqm,resale_price
floor_area_sqm,1.000000,0.667404
resale_price,0.667404,1.000000


In [17]:
resaleflat_l.head()

,floor_area_sqm,month,remaining_lease,resale_price,storey_range,flt_2-room,flt_Adjoined flat,flt_Apartment,flt_DBSS,flt_Improved,...,st_YISHUN ST 71,st_YISHUN ST 72,st_YISHUN ST 81,st_YUAN CHING RD,st_YUNG AN RD,st_YUNG HO RD,st_YUNG LOH RD,st_YUNG PING RD,st_YUNG SHENG RD,st_ZION RD
0,-2.163608,0.540959,0.710285,232000.0,-0.030180,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,-1.225543,0.540959,0.681966,250000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.225543,0.540959,0.751191,262000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-1.184757,0.540959,0.738605,265000.0,-0.365163,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.225543,0.540959,0.751191,265000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
resaleflat_l.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139788 entries, 0 to 37152
Columns: 562 entries, floor_area_sqm to st_ZION RD
dtypes: float64(5), uint8(557)
memory usage: 80.7 MB


In [19]:
resaleflat_l.sum().index.values

array(['floor_area_sqm', 'month', 'remaining_lease', 'resale_price',
       'storey_range', 'flt_2-room', 'flt_Adjoined flat', 'flt_Apartment',
       'flt_DBSS', 'flt_Improved', 'flt_Improved-Maisonette',
       'flt_Maisonette', 'flt_Model A', 'flt_Model A-Maisonette',
       'flt_Model A2', 'flt_Multi Generation', 'flt_New Generation',
       'flt_Premium Apartment', 'flt_Premium Apartment Loft',
       'flt_Premium Apartment.', 'flt_Premium Maisonette',
       'flt_Simplified', 'flt_Standard', 'flt_Terrace', 'flt_Type S1',
       'flt_Type S2', 'st_ADMIRALTY DR', 'st_ADMIRALTY LINK',
       'st_AH HOOD RD', 'st_ALJUNIED CRES', 'st_ALJUNIED RD',
       'st_ANCHORVALE CRES', 'st_ANCHORVALE DR', 'st_ANCHORVALE LANE',
       'st_ANCHORVALE LINK', 'st_ANCHORVALE RD', 'st_ANG MO KIO AVE 1',
       'st_ANG MO KIO AVE 10', 'st_ANG MO KIO AVE 2',
       'st_ANG MO KIO AVE 3', 'st_ANG MO KIO AVE 4',
       'st_ANG MO KIO AVE 5', 'st_ANG MO KIO AVE 6',
       'st_ANG MO KIO AVE 8', 'st_ANG MO

In [20]:
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [21]:
#Seperate datasets

X = resaleflat_l.drop('resale_price', axis=1)
Y = resaleflat_l['resale_price']

print(X.shape)
print(Y.shape)

(139788, 561)
(139788,)


In [22]:
#Choose top 400 features for model out of 561

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X = SelectKBest(f_classif, k=400).fit_transform(X, Y)

In [23]:
print(X.shape)

(139788, 400)


In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Linear Regression

In [25]:
linReg = LinearRegression()
linReg.fit(x_train,y_train)
pred_linReg = linReg.predict(x_test)

In [26]:
pred_linReg

array([314005.24205482, 476743.94637444, 370693.22083094, ...,
       238478.76841561, 408044.47695634, 423508.12185725])

In [27]:
y_test.head()

1040     308000.0
913      460000.0
13013    381000.0
40489    750000.0
10215    480000.0
Name: resale_price, dtype: float64

In [28]:
(y_test - pred_linReg).head()

1040     -6005.242055
913     -16743.946374
13013    10306.779169
40489    11206.061714
10215    -9694.035030
Name: resale_price, dtype: float64

In [29]:
#Let's see how our model performed

import math

math.sqrt(mean_squared_error(y_test, pred_linReg, multioutput='uniform_average'))

51868.70243770968

In [30]:
#Find RMSE for baseline model - i.e. take average house price

base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())

base

#RMSE of $138415 for prediction of house price using average

138415.3257061527

In [31]:
linear_diff = math.sqrt(((y_test - pred_linReg) ** 2).mean())
linear_diff

#RMSE of $51868 using linear regression

51868.70243770962

In [32]:
def RMSE(actual,predict):
    base = math.sqrt(((actual - actual.mean()) ** 2).mean())
    x = math.sqrt(((actual - predict) ** 2).mean())
    return x/base

RMSE(y_test, pred_linReg)

0.37473236560396284

In [33]:
#Get R^2 of prediction, 0.860

linReg.score(x_test,y_test)

0.8595756541688553

In [34]:
y_test.values

array([308000., 460000., 381000., ..., 290000., 310000., 420000.])

In [35]:
pred_linReg

array([314005.24205482, 476743.94637444, 370693.22083094, ...,
       238478.76841561, 408044.47695634, 423508.12185725])

In [36]:
result1 = np.concatenate(([y_test.values], [pred_linReg],[y_test.values - pred_linReg])).T

In [37]:
result1_df = pd.DataFrame(result1, columns=['actual','predict','difference'])

In [38]:
result1_df

,actual,predict,difference
0,308000.0,314005.242055,-6005.242055
1,460000.0,476743.946374,-16743.946374
2,381000.0,370693.220831,10306.779169
3,750000.0,738793.938286,11206.061714
4,480000.0,489694.035030,-9694.035030
5,325000.0,350269.120466,-25269.120466
6,385000.0,357052.301986,27947.698014
7,360000.0,405115.447285,-45115.447285
8,350000.0,348647.830456,1352.169544
9,337000.0,350079.238887,-13079.238887


In [39]:
print(result1_df['difference'].mean())
print(result1_df['difference'].max())
print(result1_df['difference'].min())

-38.51929928280582
385825.9046530149
-408042.59265664127


In [40]:
result1_df[result1_df['difference'] > 100000]

,actual,predict,difference
54,425000.0,319979.768321,105020.231679
103,818000.0,570700.252157,247299.747843
113,580000.0,467307.762197,112692.237803
182,920000.0,741593.189900,178406.810100
187,910000.0,780867.857048,129132.142952
194,890000.0,766493.183025,123506.816975
212,625000.0,520734.463915,104265.536085
232,808000.0,593818.903365,214181.096635
253,670000.0,564311.868604,105688.131396
291,855000.0,754723.572461,100276.427539


In [41]:
result1_df[result1_df['difference'] < -100000]

,actual,predict,difference
75,450000.0,553569.596078,-103569.596078
76,400000.0,549088.461962,-149088.461962
78,330000.0,438745.594818,-108745.594818
178,385000.0,521155.945847,-136155.945847
198,373000.0,572778.568913,-199778.568913
211,320000.0,426465.715131,-106465.715131
226,323000.0,423183.343989,-100183.343989
227,523000.0,635965.122290,-112965.122290
246,472000.0,582316.191920,-110316.191920
315,350000.0,518333.133138,-168333.133138


In [42]:
print(linReg.intercept_) #.coef_ for all coefficients
#Base price is $523769

523769.25604362617
